In [5]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
bd = pd.read_csv('credit-c61fbdb3-e0af-4b7b-9301-6ba7a994eeca.csv')

In [3]:
names = {'колонка_0' : 'approve_dttm', 'колонка_1' : 'utm_source', 'колонка_2' : 'specialty', 'колонка_3' : 'util_dttm', 'колонка_4' : 'semestr_cost_amt', 'колонка_5' : 'pid', 'колонка_6' : 'score', 'колонка_7' : 'marketing_flag', 'колонка_8' : 'approve_flg', 'колонка_9' : 'semestr_cnt', 'колонка_10' : 'short_nm', 'колонка_11' : 'initial_approved_amt', 'колонка_12' : 'reject_reason', 'колонка_13' : 'gender_cd', 'колонка_14' : 'initial_term', 'колонка_15' : 'name', 'колонка_16' : 'util_flg', 'колонка_17' : 'education_level_code', 'колонка_18' : 'age', 'колонка_19' : 'subside_rate', 'колонка_20' : 'create_dttm'}

bd.rename(columns=names, inplace=True)

In [6]:
df = bd[['name', 'create_dttm', 'pid', 'gender_cd', 'age', 'short_nm', 'education_level_code', 'specialty', 'semestr_cnt', 'subside_rate', 'semestr_cost_amt', 'initial_approved_amt', 'initial_term', 'utm_source', 'marketing_flag', 'approve_dttm', 'approve_flg', 'reject_reason', 'util_dttm', 'util_flg', 'score']]
df.drop(['name', 'pid'], axis=1, inplace=True)
universes = {'Московский государственный университет имени М.В. Ломоносова (МГУ)' : 'МГУ', 'Санкт-Петербургский государственный университет (СПбГУ)' : 'СПбГУ', 'Национальный исследовательский университет «Высшая школа экономики» (ВШЭ)' : 'ВШЭ', 'Московский физико-технический институт (МФТИ)' : 'МФТИ', 'Новосибирский государственный университет (НГУ)' : 'НГУ', 'Российский государственный университет нефти и газа имени И.М. Губкина' : 'РГУ НИГ', 'Казанский федеральный университет (КФУ)' : 'КФУ', 'Национальный исследовательский ядерный университет «МИФИ»' : 'МИФИ', 'Томский государственный университет (ТГУ)' : 'ТГУ', 'Пермский государственный национальный исследовательский университет (ПГНИУ)' : 'ПГНИУ', 'Красноярский государственный университет (КГУ)' : 'КГУ', 'Уральский федеральный университет имени первого Президента России Б.Н. Ельцина (УрФУ)' : 'УрФУ', 'Сибирский федеральный университет (СФУ)' : 'СФУ', 'Дальневосточный федеральный университет (ДВФУ)' : 'ДВФУ'}
df['short_nm'].replace(universes, inplace=True)
# df['sum'] = df['initial_approved_amt'] * (1+df['subside_rate'])**df['initial_term']
df.drop('reject_reason', axis=1, inplace=True)

df['util_dttm'] = pd.to_datetime(df['util_dttm']).fillna(pd.Timestamp(0)).astype(int)/1e8
df['approve_dttm'] = pd.to_datetime(df['approve_dttm']).fillna(pd.Timestamp(0)).astype(int)/1e8
df['create_dttm'] = pd.to_datetime(df['create_dttm']).fillna(pd.Timestamp(0)).astype(int)/1e8
# df['approve'] = df['approve_dttm']*df['approve_flg']

# df['t2'] = abs(df['create_dttm'] - df['approve_dttm'])

y = df['score']
df.drop(['score'], axis=1, inplace=True)
# df = df[['approve_flg', 'initial_approved_amt', 'short_nm', 'education_level_code', 'age', 'utm_source', 'gender_cd', 'subside_rate']]

In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

X = df
y = y

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Предобработка данных
# Определяем числовые и категориальные колонки
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

# Создаем трансформер для предобработки
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),  # Нормализация численных признаков
        ('cat', OneHotEncoder(), categorical_features)   # Кодирование категориальных признаков
    ])

# Создаем пайплайн для трансформации данных
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Применяем предобработку к данным
X_train_transformed = pipeline.fit_transform(X_train)
X_test_transformed = pipeline.transform(X_test)

# Если y_train является одномерным массивом, то мы можем использовать:
num_classes = y_train.shape[1] if len(y_train.shape) > 1 else len(np.unique(y))

# Создание модели MLP
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_transformed.shape[1],)),  # Входной слой
    layers.Dense(64, activation='relu'),  # Скрытый слой
    layers.Dense(num_classes, activation='softmax')  # Выходной слой
])

# Компиляция модели
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Обучение модели
model.fit(X_train_transformed, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Оценка модели
loss, accuracy = model.evaluate(X_test_transformed, y_test)
print(f"Точность на тестовых данных: {accuracy:.2f}")


Epoch 1/10


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 1), output.shape=(None, 52)

In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Предположим, что df уже загружен
# df = pd.read_csv('your_data.csv')

# Определяем целевую переменную и признаки

X = df
y = y

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Предобработка данных
# Определяем числовые и категориальные колонки
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

# Создаем трансформер для предобработки
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),  # Нормализация численных признаков
        ('cat', OneHotEncoder(), categorical_features)   # Кодирование категориальных признаков
    ])

# Создаем пайплайн для трансформации данных
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Применяем предобработку к данным
X_train_transformed = pipeline.fit_transform(X_train)
X_test_transformed = pipeline.transform(X_test)

# Проверяем целевую переменную
print("Уникальные классы:", np.unique(y))

# Убедитесь, что y - это одномерный массив меток классов
y_train = y_train.values.flatten()  # Преобразуем в одномерный массив
y_test = y_test.values.flatten()    # Преобразуем в одномерный массив

# Преобразуем целевую переменную в формат one-hot
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

# Определяем количество классов
num_classes = y_train.shape[1]  # Должно быть равно количеству уникальных классов

# Создание модели MLP
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_transformed.shape[1],)),  # Входной слой
    layers.Dense(64, activation='relu'),  # Скрытый слой
    layers.Dense(num_classes, activation='softmax')  # Выходной слой
])

# Компиляция модели
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Обучение модели
model.fit(X_train_transformed, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Оценка модели
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Точность на тестовых данных: {accuracy:.2f}")


Уникальные классы: [0.44 0.45 0.46 0.47 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56 0.57
 0.58 0.59 0.6  0.61 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.71
 0.72 0.73 0.74 0.75 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84 0.85
 0.86 0.87 0.88 0.89 0.9  0.91 0.92 0.93 0.94 0.95]
Epoch 1/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
160/160 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
160/160 ━━━━━━━━━━━━━━━━━

ValueError: could not convert string to float: 'F'

In [ ]:
# Оценка модели
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Точность на тестовых данных: {accuracy:.2f}")

In [11]:
print(loss)

0.0


In [12]:
import shap

# Создаем объект Explainer для модели
explainer = shap.KernelExplainer(model.predict, X_train_transformed)

# Получаем SHAP значения для тестовой выборки
shap_values = explainer.shap_values(X_test_transformed)

# Визуализация SHAP значений для первого примера
shap.initjs()  # Для инициализации визуализации
shap.force_plot(explainer.expected_value, shap_values[0], X_test_transformed[0])

200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


Using 6380 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/1596 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
 76794/425865 ━━━━━━━━━━━━━━━━━━━━ 3:41 636us/step

KeyboardInterrupt: 

In [13]:

# Создаем объект Explainer для модели
explainer = shap.KernelExplainer(model.predict, X_train_transformed)

200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 873us/step


Using 6380 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


In [14]:
# Получаем SHAP значения для тестовой выборки
shap_values = explainer.shap_values(X_test_transformed)


  0%|          | 0/1596 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
425865/425865 ━━━━━━━━━━━━━━━━━━━━ 340s 798us/step


KeyboardInterrupt: 

In [15]:
# Извлечение весов
weights_1, biases_1 = model.layers[0].get_weights()  # Первый слой
weights_2, biases_2 = model.layers[1].get_weights()  # Второй слой
weights_output, biases_output = model.layers[2].get_weights()  # Выходной слой

# Посмотрим на форму весов и смещений
print("Weights Layer 1:", weights_1.shape)
print("Biases Layer 1:", biases_1.shape)
print("Weights Layer 2:", weights_2.shape)
print("Biases Layer 2:", biases_2.shape)
print("Weights Output Layer:", weights_output.shape)
print("Biases Output Layer:", biases_output.shape)


Weights Layer 1: (44, 64)
Biases Layer 1: (64,)
Weights Layer 2: (64, 64)
Biases Layer 2: (64,)
Weights Output Layer: (64, 1)
Biases Output Layer: (1,)


In [16]:
import numpy as np

def get_formula(model):
    # Извлечение весов и смещений
    weights_1, biases_1 = model.layers[0].get_weights()  # Первый слой
    weights_2, biases_2 = model.layers[1].get_weights()  # Второй слой
    weights_output, biases_output = model.layers[2].get_weights()  # Выходной слой

    # Инициализация формулы
    formula = "score ≈ "
    
    # Формируем выражения для первого скрытого слоя
    for i in range(weights_1.shape[1]):  # Для каждого нейрона в первом слое
        terms = []
        for j in range(weights_1.shape[0]):  # Для каждого входного признака
            if weights_1[j, i] != 0:
                terms.append(f"{weights_1[j, i]:.2f} * df['f{j+1}']")
        term = f" + {biases_1[i]:.2f}" if biases_1[i] != 0 else ""
        formula += " * ".join(terms) + term + " + "

    # Убираем последний символ " + "
    formula = formula[:-3] + "\n"

    # Формируем выражения для второго скрытого слоя
    for i in range(weights_2.shape[1]):  # Для каждого нейрона во втором слое
        terms = []
        for j in range(weights_2.shape[0]):  # Для каждого нейрона из предыдущего слоя
            if weights_2[j, i] != 0:
                terms.append(f"{weights_2[j, i]:.2f} * ReLU(...)")  # Здесь будет ReLU из первого слоя
        term = f" + {biases_2[i]:.2f}" if biases_2[i] != 0 else ""
        formula += " * ".join(terms) + term + " + "

    # Формируем выражения для выходного слоя
    for i in range(weights_output.shape[1]):  # Для каждого класса
        terms = []
        for j in range(weights_output.shape[0]):  # Для каждого нейрона из второго слоя
            if weights_output[j, i] != 0:
                terms.append(f"{weights_output[j, i]:.2f} * ReLU(...)")  # Здесь будет ReLU из второго слоя
        term = f" + {biases_output[i]:.2f}" if biases_output[i] != 0 else ""
        formula += " * ".join(terms) + term + " + "

    # Убираем последний символ " + "
    formula = formula[:-3]
    
    return formula

# Получаем формулу
formula = get_formula(model)
print(formula)


score ≈ 0.22 * df['f1'] * 0.12 * df['f2'] * -0.13 * df['f3'] * 0.06 * df['f4'] * -0.19 * df['f5'] * -0.15 * df['f6'] * 0.10 * df['f7'] * -0.12 * df['f8'] * -0.06 * df['f9'] * 0.01 * df['f10'] * 0.12 * df['f11'] * 0.18 * df['f12'] * -0.13 * df['f13'] * 0.21 * df['f14'] * 0.13 * df['f15'] * -0.21 * df['f16'] * 0.11 * df['f17'] * -0.22 * df['f18'] * -0.12 * df['f19'] * 0.01 * df['f20'] * 0.10 * df['f21'] * 0.07 * df['f22'] * -0.19 * df['f23'] * -0.18 * df['f24'] * -0.11 * df['f25'] * -0.18 * df['f26'] * -0.18 * df['f27'] * -0.04 * df['f28'] * -0.23 * df['f29'] * 0.19 * df['f30'] * -0.15 * df['f31'] * 0.12 * df['f32'] * -0.07 * df['f33'] * 0.08 * df['f34'] * 0.10 * df['f35'] * -0.00 * df['f36'] * 0.22 * df['f37'] * -0.13 * df['f38'] * -0.11 * df['f39'] * 0.09 * df['f40'] * 0.19 * df['f41'] * 0.13 * df['f42'] * -0.07 * df['f43'] * 0.19 * df['f44'] + -0.06 * df['f1'] * 0.05 * df['f2'] * 0.01 * df['f3'] * 0.10 * df['f4'] * -0.23 * df['f5'] * 0.14 * df['f6'] * -0.15 * df['f7'] * 0.15 * df['f8'